In [16]:
# compute kl divergence between two multivariate gaussians pytorch

import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.distributions as dist

# define two multivariate gaussians
dim = 3
batch = 10
mu1 = torch.zeros(batch, dim)
sigma1 = torch.eye(dim, dim).repeat(batch, 1, 1)
mu2 = torch.rand(batch, dim)
sigma2 = torch.rand(batch, dim, dim)
sigma2 = torch.bmm(sigma2, sigma2.transpose(1,2)) # make it positive definite

In [17]:
# create multivariate gaussians
p = dist.MultivariateNormal(mu1, sigma1)
q = dist.MultivariateNormal(mu2, sigma2)

# compute the kl divergence
kl = dist.kl_divergence(p, q).sum()
print(kl)

tensor(2025773.6250)


In [18]:
det = torch.det(sigma2)
kl_loss = 0.5 * torch.sum(torch.sum(mu2.pow(2), dim=1) + torch.einsum("...ii", sigma2) - mu2.shape[1] - torch.log(det))
print(kl_loss)

tensor(38.0896)
